In [1]:
import os
os.chdir('..')

In [20]:
import time
import json
import os
import pickle
import copy
import numpy as np
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, STATUS_FAIL

from algos import SBFGSABS
from algos import BFGS

from models import Nested
from models import MNL

import scipy.optimize as sco

data_folder = '../data/'
figures_folder = '../figures/'

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Preparation

First, we make sure that everything works. 

In [3]:
model = Nested(data_folder)

In [4]:
base_res = model.optimize(BFGS, **{'verbose': False, 'nbr_epochs': 50, 'thresh': 1e-6})

In [9]:
base_res

       fun: -5236.900013578789
      hess: array([[ 1.40370991e-03,  1.25413008e-03,  7.76423295e-05,
        -1.28628412e-03, -1.02565649e-03],
       [ 1.25413008e-03,  2.12454700e-03,  3.73311804e-04,
        -1.26586512e-03,  7.81086078e-04],
       [ 7.76423295e-05,  3.73311804e-04,  2.19325510e-03,
         1.00240644e-03,  2.53225307e-03],
       [-1.28628412e-03, -1.26586512e-03,  1.00240644e-03,
         3.21335967e-03,  3.82563033e-03],
       [-1.02565649e-03,  7.81086078e-04,  2.53225307e-03,
         3.82563033e-03,  1.35905116e-02]])
       jac: array([ 4.36893862e-05, -1.52436359e-05, -2.76837658e-06,  1.16726683e-05,
        7.74149458e-06])
       nep: 15
       nit: 15
 opti_time: 7.657468
    status: 'Optimum reached!'
   success: True
         x: array([-0.16715561, -0.51194801, -0.85666526, -0.89866378,  2.05406558])

In [10]:
fprime = lambda x: -model.biogeme.calculateLikelihoodAndDerivatives(x, hessian=False)[1]
f = lambda x: -model.biogeme.calculateLikelihood(x)

In [17]:
res = sco.minimize(f, model.x0, jac=fprime, bounds=model.bounds)
res

      fun: 5236.90001366234
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.00142948, -0.01411452,  0.00409671, -0.00873904,  0.00266863])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 17
      nit: 15
   status: 0
  success: True
        x: array([-0.1671623 , -0.51196111, -0.85666346, -0.89866183,  2.05406615])

Define the space

In [15]:
space = {'ftol': hp.quniform('window', 1e-6, 1e-5, 1e-7),
         'eps': hp.quniform('count_upd', 1e-8, 1e-7, 1e-9),
         'maxiter': hp.quniform('factor_upd', 10, 30, 1)}

## Objective function

We define the objective function required for hyperopt.

In [31]:
def objective(model, params, base_val, data_folder):
    ftol = params['ftol']
    eps = params['eps']
    maxiter = params['maxiter']
    
    epochs = []
    fval = []
    for i in range(5):
        model = Nested(data_folder)
        
        fprime = lambda x: -model.biogeme.calculateLikelihoodAndDerivatives(x, hessian=False)[1]
        f = lambda x: -model.biogeme.calculateLikelihood(x)
        
        res = sco.minimize(f, model.x0, jac=fprime, bounds=model.bounds, options={'ftol': ftol, 'eps': eps, 'maxiter': maxiter})

        epochs.append(res.nit)
        fval.append(res.fun)

    if np.abs(np.mean(fval) - base_val) < 1e-5:
        status = STATUS_OK
    else:
        status = STATUS_FAIL

    return {'loss': np.max(epochs), 'value': np.mean(fval), 'status': status, 'epochs': epochs}


In [ ]:
trials = Trials()
best = fmin(fn=lambda x: objective(model, x, res.fun, data_folder),
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

01-04-2019:17:28:17,640 INFO     [tpe.py:837] tpe_transform took 0.000912 seconds
01-04-2019:17:28:17,640 INFO     [tpe.py:867] TPE using 0 trials
01-04-2019:17:28:34,515 INFO     [tpe.py:837] tpe_transform took 0.000893 seconds
01-04-2019:17:28:34,516 INFO     [tpe.py:865] TPE using 1/1 trials with best loss 12.000000
01-04-2019:17:28:54,695 INFO     [tpe.py:837] tpe_transform took 0.001539 seconds
01-04-2019:17:28:54,696 INFO     [tpe.py:865] TPE using 2/2 trials with best loss 12.000000
01-04-2019:17:29:16,103 INFO     [tpe.py:837] tpe_transform took 0.001282 seconds
01-04-2019:17:29:16,103 INFO     [tpe.py:865] TPE using 3/3 trials with best loss 12.000000
01-04-2019:17:29:36,841 INFO     [tpe.py:837] tpe_transform took 0.000957 seconds
01-04-2019:17:29:36,842 INFO     [tpe.py:865] TPE using 4/4 trials with best loss 12.000000
01-04-2019:17:29:57,56 INFO     [tpe.py:837] tpe_transform took 0.001101 seconds
01-04-2019:17:29:57,57 INFO     [tpe.py:865] TPE using 5/5 trials with best 

01-04-2019:17:46:00,256 INFO     [tpe.py:837] tpe_transform took 0.000988 seconds
01-04-2019:17:46:00,256 INFO     [tpe.py:865] TPE using 47/47 trials with best loss 10.000000
01-04-2019:17:46:17,735 INFO     [tpe.py:837] tpe_transform took 0.001003 seconds
01-04-2019:17:46:17,736 INFO     [tpe.py:865] TPE using 48/48 trials with best loss 10.000000
01-04-2019:17:46:34,864 INFO     [tpe.py:837] tpe_transform took 0.001294 seconds
01-04-2019:17:46:34,864 INFO     [tpe.py:865] TPE using 49/49 trials with best loss 10.000000
01-04-2019:17:46:52,236 INFO     [tpe.py:837] tpe_transform took 0.000955 seconds
01-04-2019:17:46:52,237 INFO     [tpe.py:865] TPE using 50/50 trials with best loss 10.000000
01-04-2019:17:47:09,696 INFO     [tpe.py:837] tpe_transform took 0.001005 seconds
01-04-2019:17:47:09,697 INFO     [tpe.py:865] TPE using 51/51 trials with best loss 10.000000
01-04-2019:17:47:26,347 INFO     [tpe.py:837] tpe_transform took 0.001090 seconds
01-04-2019:17:47:26,348 INFO     [tpe.

01-04-2019:17:59:50,140 INFO     [tpe.py:837] tpe_transform took 0.000928 seconds
01-04-2019:17:59:50,141 INFO     [tpe.py:865] TPE using 94/94 trials with best loss 10.000000
